### Importing packages

In [1]:
#Importing packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, f1_score,confusion_matrix
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

### LOading Data

In [2]:
# load train data
df = pd.read_csv(r"C:\Users\Triloki-Gupta\Downloads\Training Data.csv")
print(df.shape)
df.head()

(180212, 11)


,patient_id,Age,Gender,Prescription_period,Diabetes,Alcoholism,HyperTension,Smokes,Tuberculosis,Sms_Reminder,Adherence
0,1,19,M,7,0,0,0,0,0,0,No
1,2,24,F,59,0,0,0,0,0,0,No
2,3,4,F,43,0,0,0,0,0,0,No
3,4,38,M,66,0,0,0,0,0,1,No
4,5,46,F,98,0,0,0,0,0,1,No


In [3]:
#loading test data
test_df = pd.read_csv(r"C:\Users\Triloki-Gupta\Downloads\Test Data.csv")
print(test_df.shape)
test_df.head()

(119788, 10)


,patient_id,Age,Gender,Prescription_period,Diabetes,Alcoholism,HyperTension,Smokes,Tuberculosis,Sms_Reminder
0,1,5,M,28,0,0,0,0,0,1
1,2,62,F,9,1,0,1,0,0,0
2,3,4,F,73,0,0,0,0,0,1
3,4,33,M,117,0,0,0,0,0,0
4,5,38,M,8,0,0,0,0,0,1


### EDA

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180212 entries, 0 to 180211
Data columns (total 11 columns):
patient_id             180212 non-null int64
Age                    180212 non-null int64
Gender                 180212 non-null object
Prescription_period    180212 non-null int64
Diabetes               180212 non-null int64
Alcoholism             180212 non-null int64
HyperTension           180212 non-null int64
Smokes                 180212 non-null int64
Tuberculosis           180212 non-null int64
Sms_Reminder           180212 non-null int64
Adherence              180212 non-null object
dtypes: int64(9), object(2)
memory usage: 15.1+ MB


In [5]:
df.isnull().sum()

patient_id             0
Age                    0
Gender                 0
Prescription_period    0
Diabetes               0
Alcoholism             0
HyperTension           0
Smokes                 0
Tuberculosis           0
Sms_Reminder           0
Adherence              0
dtype: int64

In [6]:
df.describe()

,patient_id,Age,Prescription_period,Diabetes,Alcoholism,HyperTension,Smokes,Tuberculosis,Sms_Reminder
count,180212.000000,180212.000000,180212.000000,180212.000000,180212.000000,180212.000000,180212.000000,180212.000000,180212.000000
mean,90106.500000,37.795363,54.668485,0.078524,0.025043,0.216512,0.052566,0.000338,0.573968
std,52022.867693,22.852072,35.752491,0.268995,0.156255,0.411868,0.223166,0.018395,0.499824
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,45053.750000,19.000000,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,90106.500000,38.000000,51.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,135159.250000,56.000000,86.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,180212.000000,113.000000,120.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [7]:
df.select_dtypes(include = ['object']).head(3)

,Gender,Adherence
0,M,No
1,F,No
2,F,No


In [8]:
df.select_dtypes(include = ['int64','float64']).head(3)

,patient_id,Age,Prescription_period,Diabetes,Alcoholism,HyperTension,Smokes,Tuberculosis,Sms_Reminder
0,1,19,7,0,0,0,0,0,0
1,2,24,59,0,0,0,0,0,0
2,3,4,43,0,0,0,0,0,0


In [9]:
df.Adherence.value_counts()

No     125822
Yes     54390
Name: Adherence, dtype: int64

In [11]:
df.Gender.value_counts()

F    120438
M     59774
Name: Gender, dtype: int64

In [16]:
data.corr()

,Age,Gender,Prescription_period,Diabetes,Alcoholism,HyperTension,Smokes,Tuberculosis,Sms_Reminder,Adherence
Age,1.000000,-0.102932,0.061055,0.295050,0.071761,0.509416,0.090797,0.001300,-0.051713,-0.100818
Gender,-0.102932,1.000000,-0.004764,-0.019222,0.094593,-0.047295,0.040503,0.008821,-0.030121,0.010232
Prescription_period,0.061055,-0.004764,1.000000,0.017072,-0.015018,0.035913,-0.016020,-0.001677,0.001959,-0.612656
Diabetes,0.295050,-0.019222,0.017072,1.000000,0.021997,0.423884,0.041240,0.004721,-0.031252,-0.031004
Alcoholism,0.071761,0.094593,-0.015018,0.021997,1.000000,0.068796,0.390472,0.045315,-0.017998,0.024982
HyperTension,0.509416,-0.047295,0.035913,0.423884,0.068796,1.000000,0.077154,-0.001617,-0.043751,-0.061314
Smokes,0.090797,0.040503,-0.016020,0.041240,0.390472,0.077154,1.000000,0.021349,-0.011701,0.023993
Tuberculosis,0.001300,0.008821,-0.001677,0.004721,0.045315,-0.001617,0.021349,1.000000,-0.006043,0.002359
Sms_Reminder,-0.051713,-0.030121,0.001959,-0.031252,-0.017998,-0.043751,-0.011701,-0.006043,1.000000,-0.002519
Adherence,-0.100818,0.010232,-0.612656,-0.031004,0.024982,-0.061314,0.023993,0.002359,-0.002519,1.000000


### Preprocessing

In [4]:
test = test_df.copy()
test['Gender']=test['Gender'].apply(lambda x: 1 if x=='M' else 0)
test = test.drop(['patient_id'], axis=1)

In [5]:
df['Adherence']=df['Adherence'].apply(lambda x: 1 if x=='Yes' else 0)
df['Gender']=df['Gender'].apply(lambda x: 1 if x=='M' else 0)
data = df.drop(['patient_id'], axis=1)
X=data.drop('Adherence',axis=1)
y=data.Adherence

In [7]:
# splinting data into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=42, shuffle=False)

### ML Model

#### XGBClassifier

***Parameter tuning***

In [103]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
xgb=XGBClassifier()
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'learning_rate':[0.01,0.1,1,10]
        }
CV_xgb = GridSearchCV(estimator=xgb, param_grid=params, cv= 3)
CV_xgb.fit(X, y)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5], 'learning_rate': [0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [104]:
CV_xgb.best_params_

{'colsample_bytree': 0.6,
 'gamma': 0.5,
 'learning_rate': 0.01,
 'max_depth': 3,
 'min_child_weight': 1,
 'subsample': 0.6}

In [9]:
%%time
xgb = XGBClassifier(colsample_bytree=0.6,gamma=0.5,learning_rate=0.01,max_depth=3,min_child_weight=1,subsample=0.6)
xgb.fit(X, y)
y_pred=xgb.predict(test)
# print("classification_report")
# print(classification_report(y_test,y_pred))
# print('Confusion_matrix : ')
# print(confusion_matrix(y_test,y_pred))
# print("accuracy_score")
# print(accuracy_score(y_test,y_pred))

Wall time: 17.3 s


In [14]:
%%time
prob = pd.DataFrame(xgb.predict_proba(test), columns=["prob0","Probability_Score"])
adhe = pd.DataFrame(xgb.predict(test), columns=["Adherence"])
adhe['Adherence'] = adhe['Adherence'].apply(lambda x: 'Yes' if x==1 else 'No')
patient_id = test_df.patient_id
output = pd.concat([patient_id,adhe, prob.Probability_Score], axis=1)
print(output.shape)
print(output.head())
output.to_csv(r'C:\Users\Triloki-Gupta\Downloads\Results.csv',index=False)

(119788, 3)
   patient_id Adherence  Probability_Score
0           1       Yes           0.583873
1           2       Yes           0.528068
2           3        No           0.280868
3           4        No           0.283300
4           5       Yes           0.580708
Wall time: 2.37 s


#### RandomForestClassifier

***Parameter tuning***

In [83]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 2)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [200, 500], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [84]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 8,
 'max_features': 'auto',
 'n_estimators': 500}

In [107]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3)
CV_rfc.fit(X, y)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [100, 200, 500], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [108]:
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 4,
 'max_features': 'auto',
 'n_estimators': 100}

In [123]:
%%time
rf = RandomForestClassifier(criterion='gini',
 max_depth=4,
 max_features='auto',
 n_estimators=100)
rf.fit(X, y)
# y_pred=clf2.predict(X_test)
# print("classification_report")
# print(classification_report(y_test,y_pred))
# print('Confusion_matrix : ')
# print(confusion_matrix(y_test,y_pred))
# print("accuracy_score")
# print(accuracy_score(y_test,y_pred))

Wall time: 4.21 s
